In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_chuko_mansions():
    base_url = 'https://www.homes.co.jp/mansion/chuko/tokyo/23ku/list/'
    max_page = 188  # 最大ページ数
    mansions = []

    for page in range(1, max_page):
        url = f'{base_url}?page={page}'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        for mansion in soup.select('.moduleInner'):
            try:
                name = mansion.select_one('.bukkenName').text.strip()

                # 所在地を取得
                location_parts = mansion.select('td')[0].text.strip()

                # 価格を取得
                price_text = mansion.select_one('.priceLabel').text.strip()
                price = int(price_text.replace('万円', '').replace(',', '')) * 10000

                # 間取りと専有面積を分けて取得
                layout_and_area = mansion.select_one('.layoutSpace').text.strip()
                layout, area = layout_and_area.split('/')

                # 築年数を取得
                construction_year_text = mansion.select('td')[2].text.strip()
                construction_year = construction_year_text.split('（')[1].split('）')[0]

                mansions.append({
                    'name': name,
                    'location_parts': location_parts,
                    'price': price,
                    'layout': layout.strip(),
                    'area': area.strip(),
                    'construction_year': construction_year
                })
            except AttributeError:
                continue

    df = pd.DataFrame(mansions)
    return df

# マンション情報を取得
mansion_data = scrape_chuko_mansions()
print(mansion_data)


                  name                         location_parts      price  \
0            カーサフェリス洗足         東急目黒線 洗足駅 徒歩10分東京都品川区荏原7丁目9-16   79900000   
1     ライオンズマンション西麻布シティ     東京メトロ日比谷線 六本木駅 徒歩9分東京都港区西麻布2丁目13-8  113900000   
2          イニシアイオ新宿夏目坂       都営大江戸線 若松河田駅 徒歩6分東京都新宿区原町3丁目64-3   95800000   
3          クレストコート杉並宮前     京王井の頭線 富士見ヶ丘駅 徒歩11分東京都杉並区宮前1丁目15-1   74800000   
4         パークハウス高円寺南桃園  東京メトロ丸ノ内線 東高円寺駅 徒歩8分東京都杉並区高円寺南5丁目8-12   82900000   
...                ...                                    ...        ...   
5605            プラウド松濤          東京メトロ半蔵門線 渋谷駅 徒歩8分東京都渋谷区松濤1丁目  447800000   
5606         ベルデコリナ本駒込    東京メトロ南北線 本駒込駅 徒歩9分東京都文京区本駒込2丁目27-18  209900000   
5607    レグノ・グランデ一番町タワー      東京メトロ半蔵門線 半蔵門駅 徒歩2分東京都千代田区一番町13-6  289900000   
5608    パークコート虎ノ門愛宕タワー         東京メトロ日比谷線 神谷町駅 徒歩3分東京都港区虎ノ門3丁目  250000000   
5609   JR山手線 浜松町駅 徒歩6分               JR山手線 浜松町駅 徒歩6分東京都港区芝1丁目  698000000   

     layout      area construction_year  
0      3LDK   60.32m²              築22年  
1  

In [30]:
import re

# 正規表現を使用して情報を抽出
mansion_data["transportation"] = mansion_data["location_parts"].apply(lambda x: re.search(r"(.*) 徒歩(\d+)分", x).group(1) if isinstance(x, str) else None)
mansion_data["minutes"] = mansion_data["location_parts"].apply(lambda x: re.search(r"徒歩(\d+)分", x).group(1) if isinstance(x, str) else None)
mansion_data["address"] = mansion_data["location_parts"].apply(lambda x: re.search(r"徒歩\d+分(.*)", x).group(1) if isinstance(x, str) else None)

# 「m²」を削除して数値に変換
mansion_data['area'] = mansion_data['area'].apply(lambda x: float(x.replace('m²', '')) if isinstance(x, str) else None)

# 所在地から「東京都」の後から「○○区」までを抜き出す関数
def extract_district(address):
    match = re.search(r'東京都(.+?区)', address)
    if match:
        return match.group(1)
    else:
        return None

# 区という項目に出力
mansion_data['区'] = mansion_data['address'].apply(extract_district)

mansion_data

,name,location_parts,price,layout,area,construction_year,transportation,minutes,address,区
0,カーサフェリス洗足,東急目黒線 洗足駅 徒歩10分東京都品川区荏原7丁目9-16,79900000,3LDK,60.32,築22年,東急目黒線 洗足駅,10,東京都品川区荏原7丁目9-16,品川区
1,ライオンズマンション西麻布シティ,東京メトロ日比谷線 六本木駅 徒歩9分東京都港区西麻布2丁目13-8,113900000,2LDK,56.46,築25年,東京メトロ日比谷線 六本木駅,9,東京都港区西麻布2丁目13-8,港区
2,イニシアイオ新宿夏目坂,都営大江戸線 若松河田駅 徒歩6分東京都新宿区原町3丁目64-3,95800000,2LDK,59.13,築15年,都営大江戸線 若松河田駅,6,東京都新宿区原町3丁目64-3,新宿区
3,クレストコート杉並宮前,京王井の頭線 富士見ヶ丘駅 徒歩11分東京都杉並区宮前1丁目15-1,74800000,3LDK,75.60,築28年,京王井の頭線 富士見ヶ丘駅,11,東京都杉並区宮前1丁目15-1,杉並区
4,パークハウス高円寺南桃園,東京メトロ丸ノ内線 東高円寺駅 徒歩8分東京都杉並区高円寺南5丁目8-12,82900000,3LDK,72.26,築23年,東京メトロ丸ノ内線 東高円寺駅,8,東京都杉並区高円寺南5丁目8-12,杉並区
...,...,...,...,...,...,...,...,...,...,...
5605,プラウド松濤,東京メトロ半蔵門線 渋谷駅 徒歩8分東京都渋谷区松濤1丁目,447800000,3LDK,148.06,築16年,東京メトロ半蔵門線 渋谷駅,8,東京都渋谷区松濤1丁目,渋谷区
5606,ベルデコリナ本駒込,東京メトロ南北線 本駒込駅 徒歩9分東京都文京区本駒込2丁目27-18,209900000,4LDK,163.91,築35年,東京メトロ南北線 本駒込駅,9,東京都文京区本駒込2丁目27-18,文京区
5607,レグノ・グランデ一番町タワー,東京メトロ半蔵門線 半蔵門駅 徒歩2分東京都千代田区一番町13-6,289900000,3LDK,90.96,築17年,東京メトロ半蔵門線 半蔵門駅,2,東京都千代田区一番町13-6,千代田区
5608,パークコート虎ノ門愛宕タワー,東京メトロ日比谷線 神谷町駅 徒歩3分東京都港区虎ノ門3丁目,250000000,2LDK,65.53,築17年,東京メトロ日比谷線 神谷町駅,3,東京都港区虎ノ門3丁目,港区


In [31]:
import gspread
from gspread_dataframe import set_with_dataframe

# Google Sheets の認証
gc = gspread.service_account(filename='spread-sheet-api-419202-59ee0bee5075.json')  # 認証情報ファイルのパスを指定

# スプレッドシートを開く（スプレッドシートの URL から ID を取得して指定）
spreadsheet_id = '1qjYieNxp0gG3fOuDNkV46DL4Q1wK8vn_kcZTiYwtVIw'
worksheet_name = '購入物件'
sh = gc.open_by_key(spreadsheet_id)
worksheet = sh.worksheet(worksheet_name)

# DataFrame をスプレッドシートに書き込む
set_with_dataframe(worksheet, mansion_data)